In [1]:
from qiskit import ClassicalRegister, QuantumRegister
from qiskit import QuantumCircuit,compile, execute,IBMQ
from qiskit import BasicAer
from qiskit.tools.visualization import circuit_drawer
from qiskit.tools.visualization import plot_histogram
from qiskit.providers.ibmq import least_busy
from math import pi
import matplotlib.pyplot as plt
import sys
from qiskit.tools.visualization import circuit_drawer
from parseRealization import *
%matplotlib inline
testDir="./tests/"

C:\Users\casa\Anaconda3\lib\site-packages\marshmallow\schema.py:364: ChangedInMarshmallow3Warning: strict=False is not recommended. In marshmallow 3.0, schemas will always be strict. See https://marshmallow.readthedocs.io/en/latest/upgrading.html#schemas-are-always-strict
  ChangedInMarshmallow3Warning


In [2]:
#select a backend
IBMQ.load_accounts()
backends = IBMQ.backends(simulator=False,filters=lambda x: x.configuration().n_qubits > 4)
least_busy =least_busy(backends)
#print(available)
#least_busy = BasicAer.get_backend('qasm_simulator')
#least_busy = IBMQ.get_backend('ibmq_16_melbourne')
couplingMap = least_busy.configuration().coupling_map
print(couplingMap)
qubitsSize = least_busy.configuration().n_qubits


[[1, 0], [1, 2], [2, 3], [4, 3], [4, 10], [5, 4], [5, 6], [5, 9], [6, 8], [7, 8], [9, 8], [9, 10], [11, 3], [11, 10], [11, 12], [12, 2], [13, 1], [13, 12]]


In [3]:
def bigFunction(fileName):
    ioClass = readCircuitInformation(fileName) 
    answers = ioClass.getKmap()
    size = ioClass.getSize()
    ioClass.readGatesFromFile(fileName)
    backend_sim = BasicAer.get_backend('qasm_simulator')
    error_count = 0
    ctg = CircuitTransitionGraph()
    print(ctg.transformCoupling(couplingMap))
    ctg.setSize(size)
    for i in range(0,2):
        #This part corresponds for part of experiment without minimal or no changes
        qr,cr,qc = ioClass.createCircuitAndSetInput(i)
        qc,qr = ctg.readGatesFromIOClass(qr,qc, ioClass)
        measureFidelityWithoutChanges(qr,cr,qc)
        
        #This part corresponds for part of experiment after significant changes
        qr,cr,qc = ioClass.createCircuitAndSetInput(i)
        ctg = fixTheStuff(ctg)
        #This one needs to comply with changes you did
        qc,qr = ctg.readFixedGatesFromCtg(qr,qc)
        measureFidelityWithChanges(qr,cr,qc)
    
def fixTheStuff(ctg):
    print("Missing connections are",ctg.getMissingConnections())
    #This one to fix the changes... fixthemissingedges connects stuff around. did not test though
    ctg.fixMissingEdges()
    print("FIxing the stuff")
    return ctg


def measureFidelityWithoutChanges(qr,cr,qc):
    least_busy = BasicAer.get_backend('qasm_simulator')
    qc.measure(qr,cr)
    job = execute(qc,least_busy,shots=200)
    result = job.result()
    print(result.get_counts())
    #plot_histogram(result.get_counts())

    
#This needs to be implemented    
def measureFidelityWithChanges(qr,cr,qc):
    #least_busy = BasicAer.get_backend('qasm_simulator')
    qc.measure(qr,cr)
    job = compile(qc,least_busy,shots=200)
    #job = execute(qc,least_busy,shots=200)
    result = job.result()
    print(result.get_counts())
    #plot_histogram(result.get_counts())


In [4]:
fileName="ham3_28"
#if testFromFile(filename) == 0:
fileName = testDir+fileName
bigFunction(fileName)

./tests/ham3_28.pla
The constants are set to .constants ---

The circuit has no garbage lines in it or file parsed incorrectly, check ioclass
The circuit has no constant inputs lines in it or file parsed incorrectly, check ioclass
['ab', 'bc', 'cd', 'de', 'ek', 'ef', 'fg', 'fj', 'gi', 'hi', 'ij', 'jk', 'dl', 'kl', 'lm', 'cm', 'bn', 'mn']
Pass manager is None
Gonna do something
Initial layout is None
Initial layout after pick None


C:\Users\casa\Anaconda3\lib\site-packages\marshmallow\schema.py:364: ChangedInMarshmallow3Warning: strict=False is not recommended. In marshmallow 3.0, schemas will always be strict. See https://marshmallow.readthedocs.io/en/latest/upgrading.html#schemas-are-always-strict
  ChangedInMarshmallow3Warning


{'000': 200}
Self skeleton is ['ab', 'bc', 'ab', 'bc', 'bc', 'ab', 'bc', 'bc', 'ab', 'ab', 'bc', 'ab', 'bc', 'bc', 'ac', 'bc']
Missing connections are [['a', 'c']]
b
a
c
d
e
k
f
g
j
i
h
l
m
n
Paths to a c
Trace is
[['a', 'b', 'n', 'm', 'c'], ['a', 'b', 'n', 'm', 'l', 'k', 'j', 'f', 'e', 'd', 'c'], ['a', 'b', 'n', 'm', 'l', 'k', 'j', 'i', 'g', 'f', 'e', 'd', 'c'], ['a', 'b', 'n', 'm', 'l', 'k', 'e', 'd', 'c'], ['a', 'b', 'n', 'm', 'l', 'd', 'c'], ['a', 'b', 'c']]
['c', 'b'] ['a', 'c']
['b', 'c'] ['a', 'c']
['a', 'c'] ['a', 'c']
replaceTo ['a', 'b', 'c']
sw a b
['a', 'b']
fixed skeleton ['ab', 'bc', 'ab', 'bc', 'bc', 'ab', 'bc', 'bc', 'ab', 'ab', 'bc', 'ab', 'bc', 'bc', 'ab', 'bc']
FIxing the stuff
Pass manager is None
Gonna do something
Initial layout is None
YEAH I AM CHECKING MATCHES COUPLING MAP
YOA REACHED THE JUICY ALGO
Iterator is 0
The possible layout is [ 0  1  2 13  3 12  4 11 10  5  9  6  8  7]
I am at node 0
The miniloop will traverse []
I am at node 1
The miniloop will trave

AttributeError: 'Qobj' object has no attribute 'result'

In [7]:
import scipy.sparse as sp
import numpy as np
cmap = np.asarray([[1, 0], [1, 2], [2, 3], [4, 3], [4, 10], [5, 4], [5, 6], [5, 9], [6, 8], [7, 8], [9, 8], [9, 10], [11, 3], [11, 10], [11, 12], [12, 2], [13, 1], [13, 12]])
device_qubits = 14
data = np.ones_like(cmap[:,0])
sp_cmap = sp.coo_matrix((data, (cmap[:, 0], cmap[:, 1])),
                        shape=(device_qubits, device_qubits)).tocsr()
print(sp_cmap)
print(sp_cmap.shape)

  (1, 0)	1
  (1, 2)	1
  (2, 3)	1
  (4, 3)	1
  (4, 10)	1
  (5, 4)	1
  (5, 6)	1
  (5, 9)	1
  (6, 8)	1
  (7, 8)	1
  (9, 8)	1
  (9, 10)	1
  (11, 3)	1
  (11, 10)	1
  (11, 12)	1
  (12, 2)	1
  (13, 1)	1
  (13, 12)	1
(14, 14)


In [9]:
np.matrix([[1, 0, 2],[0, 0, 3],[4, 5, 6]])

matrix([[1, 0, 2],
        [0, 0, 3],
        [4, 5, 6]])